# TTS 음성합성

- 학습시킨 glowtts 와 hifigan을 합성시켜 TTS 음성파일 생성


## 1. 할당된 GPU 확인

- 현재 런타임에 할당된 GPU를 확인

In [1]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: Tesla T4


## <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSUX6O-4BsthBE_xhtLuEN0CNFxzn8RY8DOJvuE-0SzEBcxCNAo7V51BuUDrUGISmMgVWs&usqp=CAU" height="20" width="22" /> 구글 드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. 환경설정 및 라이브러리 불러오기

In [3]:
!python --version                                                                 # 파이썬 버전 확인

Python 3.10.12


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/data/                          # 현재 작업 디렉토리를 변경
# !git clone --depth 1 https://github.com/coqui-ai/TTS.git -b dev            # TTS GitHub 저장소를 '--depth 1' 옵션으로 최신 버전만 가져와서 'dev' 브랜치로 클론
%cd /content/drive/My Drive/Colab Notebooks/data/TTS-dev                   # 작업 디렉토리를 클론한 TTS-dev 프로젝트 폴더로 변경

In [5]:
import os                                                                 # 운영체제와 상호 작용하기 위한 기능을 제공하는 os 모듈을 가져오기
import sys                                                                # Python 시스템과 관련된 기능을 제공하는 sys 모듈 가져오기
from pathlib import Path                                                  # 파일 경로와 관련된 작업을 수행할 수 있는 pathlib 모듈에서 Path 클래스를 가져오기

In [9]:
# 현재 작업 디렉토리를 변경
%cd /content/drive/My Drive/Colab Notebooks/data/
# g2pK GitHub 저장소를 '--depth 1' 옵션으로 최신 버전만 가져와서 메인 브랜치로 클론
!git clone --depth 1 https://github.com/sce-tts/g2pK.git
# TTS-dev 디렉토리로 이동
%cd /content/drive/My Drive/Colab Notebooks/data/TTS-dev
# TTS-dev 디렉토리에 있는 소스 코드를 pip를 사용하여 설치
!pip install -q --no-cache-dir -e .
# g2pK 디렉토리로 이동
%cd /content/drive/My Drive/Colab Notebooks/data/g2pK
# g2pK 디렉토리에 있는 의존성 패키지들을 pip를 사용하여 설치
!pip install -q --no-cache-dir "konlpy" "jamo" "nltk" "python-mecab-ko"
# g2pK 디렉토리에 있는 소스 코드를 pip를 사용하여 설치
!pip install -q --no-cache-dir -e .

/content/drive/My Drive/Colab Notebooks/data
fatal: destination path 'g2pK' already exists and is not an empty directory.
/content/drive/My Drive/Colab Notebooks/data/TTS-dev
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 285.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 246.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 272.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 280.3 MB/s eta 0:00:00
  Pre

In [11]:
# 현재 작업 디렉토리를 변경
%cd /content/drive/My Drive/Colab Notebooks/data/g2pK
import g2pk                                             # 'g2pk' 모듈을 가져와서 사용하기 위해 임포트
g2p = g2pk.G2p()                                        # g2pk 모듈에서 G2p 클래스를 인스턴스화

/content/drive/My Drive/Colab Notebooks/data/g2pK


In [12]:
!pip install pysbd                       # pysbd: Sentence Boundary Detection을 위한 패키지
!pip install coqpit                      # coqpit: 코드에서 사용되는 패키지로 가상화된 환경을 관리하는 도구
!pip install unidecode                   # unidecode: 유니코드 문자열을 ASCII로 변환
!pip install pypinyin                    # pypinyin: 한자를 기반으로 한 중국어 음성 입력을 처리하기 위한 패키지
!pip install librosa==0.8.0 numba==0.52  # librosa: 오디오 및 음악 신호 처리 / numba: 고성능 파이썬 코드를 빠르게 실행하기 위한 JIT(Just-In-Time) 컴파일러
!pip install cffi                        # cffi: C 언어와의 상호 운용성을 제공
!python setup.py develop                 # TTS 프로젝트를 개발 모드로 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 0.52.0 Requires-Python >=3.6,<3.9; 0.52.0rc3 Requires-Python >=3.6,<3.9; 0.53.0 Requires-Python >=3.6,<3.10; 0.53.0rc1.post1 Requires-Python >=3.6,<3.10; 0.53.0rc2 Requires-Python >=3.6,<3.10; 0.53.0rc3 Requires-Python >=3.6,<3.10; 0.53.1 Requires-Python >=3.6,<3.10; 0.54.0 Requires-Python >=3.7,<3.10; 0.54.0rc2 Requires-Python >=3.7,<3.10; 0.54.0rc3 Requires-Python >=3.7,<3.10; 0.54.1 Requires-Python >=3.7,<3.10
ERROR: Could not find a version that satisfies the requirement numba==0.52 (from versions: 0.1, 0.2, 0.3, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.8.0, 0.8.1, 0.9.0, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.12.1, 0.12.2, 0.13.0, 0.13.2, 0.13.3, 0.13.4, 0.14.0, 0.15.1, 0.16.0, 0.17.0, 0.18.1, 0.18.2, 0.19.1, 0.19.2, 0.20.0, 0.21.0, 0.22.0, 0.22.1, 0.23.0, 0.23.1, 0.24.0, 0.25.0, 0.26.0, 0.2

In [13]:
# 현재 작업 디렉토리를 변경
%cd /content/drive/My Drive/Colab Notebooks/data/TTS

# 필요한 모듈을 가져오
import re
import sys
from unicodedata import normalize
import IPython

# TTS 프로젝트에서 제공하는 Synthesizer 클래스를 가져오기
from TTS.utils.synthesizer import Synthesizer

# 텍스트 정규화 함수를 정의
def normalize_text(text):
    # 텍스트 좌우 공백을 제거
    text = text.strip()
    # 쉼표, 세미콜론, 콜론을 마침표로 대체
    for c in ",;:":
        text = text.replace(c, ".")
    text = remove_duplicated_punctuations(text)
    # 한글 자모음으로 변환
    text = jamo_text(text)
    # 관용구 및 영어 표현을 한글 발음으로 변환
    text = g2p.idioms(text)
    text = g2pk.english.convert_eng(text, g2p.cmu)
    text = g2pk.utils.annotate(text, g2p.mecab)
    text = g2pk.numerals.convert_num(text)
    text = re.sub("/[PJEB]", "", text)
    # 영어 알파벳을 한글 발음으로 변환
    text = alphabet_text(text)
    # 읽을 수 없는 문자를 제거
    text = normalize("NFD", text)
    text = "".join(c for c in text if c in symbols)
    text = normalize("NFC", text)
    # 좌우 공백을 다시 제거
    text = text.strip()
    if len(text) == 0:
        return ""
    # 문장부호가 '.' 혹은 '!' 혹은 '?' 일 경우, 발음 텍스트로 변환
    if text in '.!?':
        return punctuation_text(text)
    # 문장부호가 빠진 경우, 문장 끝에 '.'을 추가
    if text[-1] not in '.!?':
        text += '.'
    return text

# 중복된 문장부호를 정리하는 함수를 정의
def remove_duplicated_punctuations(text):
    text = re.sub(r"[.?!]+\?", "?", text)
    text = re.sub(r"[.?!]+!", "!", text)
    text = re.sub(r"[.?!]+\.", ".", text)
    return text

# 텍스트를 문장 단위로 나누는 함수를 정의
def split_text(text):
    text = remove_duplicated_punctuations(text)
    texts = []
    for subtext in re.findall(r'[^.!?\n]*[.!?\n]', text):
        texts.append(subtext.strip())
    return texts

# 알파벳을 한글 발음으로 변환하는 함수를 정의
def alphabet_text(text):
    text = re.sub(r"(a|A)", "에이", text)
    text = re.sub(r"(b|B)", "비", text)
    text = re.sub(r"(c|C)", "씨", text)
    text = re.sub(r"(d|D)", "디", text)
    text = re.sub(r"(e|E)", "이", text)
    text = re.sub(r"(f|F)", "에프", text)
    text = re.sub(r"(g|G)", "쥐", text)
    text = re.sub(r"(h|H)", "에이치", text)
    text = re.sub(r"(i|I)", "아이", text)
    text = re.sub(r"(j|J)", "제이", text)
    text = re.sub(r"(k|K)", "케이", text)
    text = re.sub(r"(l|L)", "엘", text)
    text = re.sub(r"(m|M)", "엠", text)
    text = re.sub(r"(n|N)", "엔", text)
    text = re.sub(r"(o|O)", "오", text)
    text = re.sub(r"(p|P)", "피", text)
    text = re.sub(r"(q|Q)", "큐", text)
    text = re.sub(r"(r|R)", "알", text)
    text = re.sub(r"(s|S)", "에스", text)
    text = re.sub(r"(t|T)", "티", text)
    text = re.sub(r"(u|U)", "유", text)
    text = re.sub(r"(v|V)", "브이", text)
    text = re.sub(r"(w|W)", "더블유", text)
    text = re.sub(r"(x|X)", "엑스", text)
    text = re.sub(r"(y|Y)", "와이", text)
    text = re.sub(r"(z|Z)", "지", text)
    return text

# 문장부호를 한글 발음으로 변환하는 함수를 정의
def punctuation_text(text):
    text = re.sub(r"!", "느낌표", text)
    text = re.sub(r"\?", "물음표", text)
    text = re.sub(r"\.", "마침표", text)
    return text

# 한글 자모음을 발음으로 변환하는 함수를 정의
def jamo_text(text):
    text = re.sub(r"ㄱ", "기역", text)
    text = re.sub(r"ㄴ", "니은", text)
    text = re.sub(r"ㄷ", "디귿", text)
    text = re.sub(r"ㄹ", "리을", text)
    text = re.sub(r"ㅁ", "미음", text)
    text = re.sub(r"ㅂ", "비읍", text)
    text = re.sub(r"ㅅ", "시옷", text)
    text = re.sub(r"ㅇ", "이응", text)
    text = re.sub(r"ㅈ", "지읒", text)
    text = re.sub(r"ㅊ", "치읓", text)
    text = re.sub(r"ㅋ", "키읔", text)
    text = re.sub(r"ㅌ", "티읕", text)
    text = re.sub(r"ㅍ", "피읖", text)
    text = re.sub(r"ㅎ", "히읗", text)
    text = re.sub(r"ㄲ", "쌍기역", text)
    text = re.sub(r"ㄸ", "쌍디귿", text)
    text = re.sub(r"ㅃ", "쌍비읍", text)
    text = re.sub(r"ㅆ", "쌍시옷", text)
    text = re.sub(r"ㅉ", "쌍지읒", text)
    text = re.sub(r"ㄳ", "기역시옷", text)
    text = re.sub(r"ㄵ", "니은지읒", text)
    text = re.sub(r"ㄶ", "니은히읗", text)
    text = re.sub(r"ㄺ", "리을기역", text)
    text = re.sub(r"ㄻ", "리을미음", text)
    text = re.sub(r"ㄼ", "리을비읍", text)
    text = re.sub(r"ㄽ", "리을시옷", text)
    text = re.sub(r"ㄾ", "리을티읕", text)
    text = re.sub(r"ㄿ", "리을피읍", text)
    text = re.sub(r"ㅀ", "리을히읗", text)
    text = re.sub(r"ㅄ", "비읍시옷", text)
    text = re.sub(r"ㅏ", "아", text)
    text = re.sub(r"ㅑ", "야", text)
    text = re.sub(r"ㅓ", "어", text)
    text = re.sub(r"ㅕ", "여", text)
    text = re.sub(r"ㅗ", "오", text)
    text = re.sub(r"ㅛ", "요", text)
    text = re.sub(r"ㅜ", "우", text)
    text = re.sub(r"ㅠ", "유", text)
    text = re.sub(r"ㅡ", "으", text)
    text = re.sub(r"ㅣ", "이", text)
    text = re.sub(r"ㅐ", "애", text)
    text = re.sub(r"ㅒ", "얘", text)
    text = re.sub(r"ㅔ", "에", text)
    text = re.sub(r"ㅖ", "예", text)
    text = re.sub(r"ㅘ", "와", text)
    text = re.sub(r"ㅙ", "왜", text)
    text = re.sub(r"ㅚ", "외", text)
    text = re.sub(r"ㅝ", "워", text)
    text = re.sub(r"ㅞ", "웨", text)
    text = re.sub(r"ㅟ", "위", text)
    text = re.sub(r"ㅢ", "의", text)
    return text

# 다중 문장을 정규화하는 함수를 정의
def normalize_multiline_text(long_text):
    texts = split_text(long_text)
    normalized_texts = [normalize_text(text).strip() for text in texts]
    return [text for text in normalized_texts if len(text) > 0]

# 텍스트를 합성하여 오디오를 생성하는 함수를 정의
def synthesize(text):
    wavs = synthesizer.tts(text, None, None)
    return wavs

/content/drive/My Drive/Colab Notebooks/data/TTS


In [17]:
# Synthesizer 객체를 생성하고 필요한 경로 및 설정을 전달합니다.
synthesizer = Synthesizer(
    # GlowTTS 모델에 대한 경로 및 설정
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-December-24-2023_05+08PM-3aa165a/best_model.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/glowtts-v2-December-24-2023_05+08PM-3aa165a/config.json",
    None,
    # HIFIGAN 모델에 대한 경로 및 설정
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-December-22-2023_03+20PM-3aa165a/best_model.pth.tar",
    "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/hifigan-v2-December-22-2023_03+20PM-3aa165a/config.json",
    None,
    None,
    False,
)
# 모델에서 사용되는 문자 집합을 저장
symbols = synthesizer.tts_config.characters.characters

 > Using model: glow_tts
 > Generator Model: hifigan_generator
Removing weight norm...


In [20]:
# 현재 작업 디렉토리를 변경
%cd /content/drive/My Drive/Colab Notebooks/data/complete_TTS_model
import torch
torch.save(synthesizer, 'ys_1.pth') # glowtts와 hifigan을 합성한 모델을 저장

/content/drive/My Drive/Colab Notebooks/data/complete_TTS_model


In [21]:
tts_model = torch.load('/content/drive/My Drive/Colab Notebooks/data/complete_TTS_model/ys_1.pth') # 모델 불러오기

In [38]:
import numpy as np
import scipy.io.wavfile as wavfile

# 저장하고 싶은 파일 경로 지정
output_file_path = "/content/drive/My Drive/Colab Notebooks/data/complete_TTS_model/wavs/generated_audio.wav"

# 생성하고 싶은 텍스트 작성
texts = """
안녕하세요 케이티 에이블 팔조 피움입니다
"""

# TTS 모델을 사용하여 텍스트를 음성으로 변환
wavs = [tts_model.tts(text, None, None) for text in normalize_multiline_text(texts)]

# wavs를 하나의 음성으로 합치기
combined_wav = np.concatenate(wavs)

# 완성된 음성 확인
display(Audio(combined_wav, rate=22050))

# wav 파일로 저장
wavfile.write(output_file_path, 22050, combined_wav.astype(np.int16))

 > Text splitted to sentences.
['안녕하세요 케이티 에이블 팔조 피움입니다.']
 > Processing time: 0.3183012008666992
 > Real-time factor: 0.09550595306867404


In [33]:
# 생성하고 싶은 텍스트 작성
texts = """
안녕하세요 케이티 에이블 팔조 피움입니다
"""
for text in normalize_multiline_text(texts):                         # 정규화된 텍스트를 반복해서 TTS 모델로 전달
    wav = tts_model.tts(text, None, None)                            # TTS 모델을 사용하여 텍스트를 음성으로 변환
    IPython.display.display(IPython.display.Audio(wav, rate=22050))  # IPython.display를 사용하여 음성을 재생

 > Text splitted to sentences.
['안녕하세요 케이티 에이블 팔조 피움입니다.']
 > Processing time: 0.4692354202270508
 > Real-time factor: 0.14079361278040592


In [15]:
texts = """
군자 디지털 과학 고등학교의 신성희 선생님, 장하림 선생님 만나서 반갑습니다. 저희는 피움프로젝트를 개발하는 생각이룸 팀입니다.
"""
for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))

 > Text splitted to sentences.
['군자 디지털 과학 고등학교의 신성희 선생님.', '장하림 선생님 만나서 반갑습니다.']
 > Processing time: 1.124375343322754
 > Real-time factor: 0.1624585625934861


 > Text splitted to sentences.
['저희는 피움프로젝트를 개발하는 생각이룸 팀입니다.']
 > Processing time: 0.6148312091827393
 > Real-time factor: 0.147487251550037


In [16]:
texts = """
안녕하세요 스타벅스입니다. 무엇을 주문하시겠어요?
"""
for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))

 > Text splitted to sentences.
['안녕하세요 스타벅스입니다.']
 > Processing time: 0.4070427417755127
 > Real-time factor: 0.17097097790593674


 > Text splitted to sentences.
['무엇을 주문하시겠어요?']
 > Processing time: 0.35782456398010254
 > Real-time factor: 0.16836018341928263
